<a href="https://colab.research.google.com/github/LucasFelipeNunes/Exercicios-Inteligencia-Artificial/blob/main/Atividade_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade de Inteligência Artificial

## Autoria

**Alunos:** Lucas Felipe da Silva Nunes e Luiz Gustavo Duarte Chagas

**Professor:** Cristóvão José Dias da Cunha

**Curso:** Análise e Desenvolvimento de Sistemas (6º ADS)

**Faculdade de Tecnologia de Guaratinguetá Professor João Mod**

## Escopo

Neste projeto, é analisada uma [base de dados](https://www.kaggle.com/datasets/bhadramohit/mental-health-dataset), no formato CSV, que contém informações sobre pacientes que foram registrados em um hospital, incluindo dados pessoais, profissionais e a informação de se eles tem uma condição de saúde mental ou não.

Este notebook busca analisar como pode se melhor treinar uma Inteligência Artificial para analisar esse tipo de dados e fazer predições com base neles.

## Importação das Bibliotecas

Para isso, primeiramente, importa-se as bibliotecas que serão utilizadas - incluiundo a do próprio Google Drive, em que se está armazenado o banco de dados.

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import f1_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Funções Auxiliares

Além disso, inclui-se como função auxiliar uma corretora de cores, para facilitar a visualização dos dados na tabela de correlação que será analisada posteriormente.

### Função para Colorir a Tabela de Correlação

In [4]:
def color_corr ( value_str ):
  try:
    value = float(value_str)
    if value >= 0.6 or value <= -0.6:
      color = 'red'
    else:
      color = 'blue'
  except:
          color = 'blue'
  return 'color: %s' % color

## Importação de Dados

Começando o fluxo principal do código, a primeira coisa que se faz é importar os dados da base.

In [5]:
df=pd.read_csv('/content/drive/My Drive/mental_health_dataset.csv')

Para se visualizar o formato da base de dados, pode se imprimir as suas primeiras linhas.

In [6]:
df.head()

,User_ID,Age,Gender,Occupation,Country,Mental_Health_Condition,Severity,Consultation_History,Stress_Level,Sleep_Hours,Work_Hours,Physical_Activity_Hours
0,1,36,Non-binary,Sales,Canada,No,Medium,Yes,Medium,7.1,46,5
1,2,34,Female,Education,UK,Yes,NaN,No,Low,7.5,47,8
2,3,65,Non-binary,Sales,USA,Yes,High,No,Low,8.4,58,10
3,4,34,Male,Other,Australia,No,Low,No,Medium,9.8,30,2
4,5,22,Female,Healthcare,Canada,Yes,Low,No,Medium,4.9,62,5


## Retirando Colunas Inutilzadas

Como se pode ver, a tabela contém uma coluna que não diz nada sobre o paciente ou sua condição mental: a User_ID. Portanto, pode-se removê-la.

In [7]:
df.drop(['User_ID'],axis=1,inplace=True)

Além disso, pode-se ver que a coluna **Severity** contém dados registrados como NaN (Not a Number). Para se ter uma melhor visualização disso, pode-se imprimir os valores e suas quantidades presentes em cada registro neste campo.

In [8]:
value_counts = df['Severity'].value_counts(dropna=False)
print(value_counts)

Severity
NaN       501
Low       176
Medium    164
High      159
Name: count, dtype: int64


Como, neste projeto, busca-se apenas a identificação de se a pessoa tem alguma condição de saúde mental ou não, e como a remoção de todos os registros **NaN** geraria a perda de metade do dataset, opta-se por remover a coluna de severidade.

In [9]:
df.drop(['Severity'],axis=1,inplace=True)

Após este processo, as colunas do base de dados ficam conforme se segue:

In [10]:
df.head()

,Age,Gender,Occupation,Country,Mental_Health_Condition,Consultation_History,Stress_Level,Sleep_Hours,Work_Hours,Physical_Activity_Hours
0,36,Non-binary,Sales,Canada,No,Yes,Medium,7.1,46,5
1,34,Female,Education,UK,Yes,No,Low,7.5,47,8
2,65,Non-binary,Sales,USA,Yes,No,Low,8.4,58,10
3,34,Male,Other,Australia,No,No,Medium,9.8,30,2
4,22,Female,Healthcare,Canada,Yes,No,Medium,4.9,62,5


## Transformando Colunas Categóricas em Numéricas

Também é importante se analisar as colunas categóricas: ou seja, aquelas que contém uma enumeração de valores que podem estar registradas. A intenção e transformá-las em numéricas, para facilitar a análise do modelo. Nesta base de dados, essas colunas são:

In [11]:
df.select_dtypes(include=['object'])

,Gender,Occupation,Country,Mental_Health_Condition,Consultation_History,Stress_Level
0,Non-binary,Sales,Canada,No,Yes,Medium
1,Female,Education,UK,Yes,No,Low
2,Non-binary,Sales,USA,Yes,No,Low
3,Male,Other,Australia,No,No,Medium
4,Female,Healthcare,Canada,Yes,No,Medium
...,...,...,...,...,...,...
995,Non-binary,Other,Germany,No,Yes,Low
996,Male,Sales,Canada,No,No,High
997,Female,IT,Germany,No,Yes,Medium
998,Female,Finance,Australia,Yes,No,Medium


Para se fazer este processo, primeiro atribui-se as colunas a uma variável **colunas_object**.

In [12]:
colunas_object = list(df.select_dtypes(include=['object']).columns)
colunas_object

['Gender',
 'Occupation',
 'Country',
 'Mental_Health_Condition',
 'Consultation_History',
 'Stress_Level']

### Transformando-as em Colunas Númericas (Técnica Dummie)

Depois, utiliza-se este objeto para orientar a função **get_dummies** para transformar as colunas categóricas em numéricas pela Técnica Dummie.

In [13]:
df = pd.get_dummies( df, columns = colunas_object )

Após estes processos, as colunas da tabela ficam da seguinte forma:

In [14]:
df.head()

,Age,Sleep_Hours,Work_Hours,Physical_Activity_Hours,Gender_Female,Gender_Male,Gender_Non-binary,Gender_Prefer not to say,Occupation_Education,Occupation_Engineering,...,Country_Other,Country_UK,Country_USA,Mental_Health_Condition_No,Mental_Health_Condition_Yes,Consultation_History_No,Consultation_History_Yes,Stress_Level_High,Stress_Level_Low,Stress_Level_Medium
0,36,7.1,46,5,False,False,True,False,False,False,...,False,False,False,True,False,False,True,False,False,True
1,34,7.5,47,8,True,False,False,False,True,False,...,False,True,False,False,True,True,False,False,True,False
2,65,8.4,58,10,False,False,True,False,False,False,...,False,False,True,False,True,True,False,False,True,False
3,34,9.8,30,2,False,True,False,False,False,False,...,False,False,False,True,False,True,False,False,False,True
4,22,4.9,62,5,True,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,True


## Removendo Dados com Alta Correlação

Como se pode notar, as colunas que permitem apenas o registro de valores em uma enumeração entre dois possíveis geram colunas númericas completamente inversamente correlacionadas. Desta forma, pode-se retirar uma das colunas númericas geradas neste caso. Analisando a tabela de correlação para ver quais são os casos:

In [15]:
df.corr().style.applymap(color_corr)

,Age,Sleep_Hours,Work_Hours,Physical_Activity_Hours,Gender_Female,Gender_Male,Gender_Non-binary,Gender_Prefer not to say,Occupation_Education,Occupation_Engineering,Occupation_Finance,Occupation_Healthcare,Occupation_IT,Occupation_Other,Occupation_Sales,Country_Australia,Country_Canada,Country_Germany,Country_India,Country_Other,Country_UK,Country_USA,Mental_Health_Condition_No,Mental_Health_Condition_Yes,Consultation_History_No,Consultation_History_Yes,Stress_Level_High,Stress_Level_Low,Stress_Level_Medium
Age,1.000000,-0.043114,-0.013825,0.052679,0.017281,0.009409,0.009036,-0.038218,0.018349,-0.034479,0.029596,0.027362,0.000077,-0.027780,-0.011192,0.065866,-0.023269,0.039544,-0.004404,-0.019879,-0.010138,-0.051366,-0.010689,0.010689,0.024420,-0.024420,0.014630,0.020963,-0.036135
Sleep_Hours,-0.043114,1.000000,-0.005550,-0.015012,0.055211,-0.022082,-0.035398,0.001632,0.013656,0.012907,-0.003355,-0.041877,-0.010684,0.037975,-0.010260,0.021981,0.002720,0.021673,0.035135,-0.011883,-0.037779,-0.034522,0.026555,-0.026555,0.030150,-0.030150,0.028187,0.038735,-0.067944
Work_Hours,-0.013825,-0.005550,1.000000,0.039258,-0.034575,-0.041226,0.075984,-0.001180,0.008592,-0.006304,0.019402,-0.029709,0.040143,0.024803,-0.057293,-0.028640,0.009131,-0.054053,0.016867,-0.039959,0.023138,0.069078,-0.037317,0.037317,0.036106,-0.036106,0.013280,0.029780,-0.043704
Physical_Activity_Hours,0.052679,-0.015012,0.039258,1.000000,-0.034501,-0.012871,0.019243,0.030023,0.017015,-0.016303,-0.029679,-0.017291,0.032676,-0.003157,0.018915,0.009351,-0.011758,0.023420,-0.017735,0.034182,0.015366,-0.048258,-0.008438,0.008438,0.030733,-0.030733,-0.040260,0.054705,-0.014532
Gender_Female,0.017281,0.055211,-0.034575,-0.034501,1.000000,-0.348314,-0.367049,-0.319219,-0.002966,0.012940,0.022593,0.042824,-0.039202,0.009377,-0.049106,-0.013517,-0.060475,0.057797,-0.036410,0.042010,-0.009962,0.024845,-0.022309,0.022309,0.011939,-0.011939,0.012630,-0.082188,0.070499
Gender_Male,0.009409,-0.022082,-0.041226,-0.012871,-0.348314,1.000000,-0.345664,-0.300621,-0.036268,-0.010160,-0.008935,0.007795,0.021503,-0.004839,0.031586,0.022011,0.026314,-0.012173,-0.014640,-0.053822,0.037984,-0.009972,-0.008328,0.008328,-0.012684,0.012684,0.061227,-0.021990,-0.039965
Gender_Non-binary,0.009036,-0.035398,0.075984,0.019243,-0.367049,-0.345664,1.000000,-0.316791,0.039391,0.015812,-0.039943,-0.049406,0.023225,0.024681,-0.013527,0.001726,0.020671,-0.036678,0.035071,0.037518,-0.039943,-0.016269,0.011330,-0.011330,-0.021860,0.021860,-0.030086,0.080061,-0.050590
Gender_Prefer not to say,-0.038218,0.001632,-0.001180,0.030023,-0.319219,-0.300621,-0.316791,1.000000,-0.001138,-0.020311,0.027929,-0.001256,-0.005210,-0.031578,0.034418,-0.010340,0.015444,-0.010166,0.016921,-0.029250,0.013880,0.001137,0.020616,-0.020616,0.023913,-0.023913,-0.045448,0.025645,0.020212
Occupation_Education,0.018349,0.013656,0.008592,0.017015,-0.002966,-0.036268,0.039391,-0.001138,1.000000,-0.164653,-0.158732,-0.165305,-0.154730,-0.173058,-0.156069,-0.020754,0.003139,-0.017111,0.041036,-0.004816,-0.023389,0.020214,0.014785,-0.014785,0.034094,-0.034094,0.005120,0.020848,-0.026348
Occupation_Engineering,-0.034479,0.012907,-0.006304,-0.016303,0.012940,-0.010160,0.015812,-0.020311,-0.164653,1.000000,-0.167462,-0.174397,-0.163240,-0.182576,-0.164653,-0.020587,0.012868,-0.023207,0.016030,-0.044313,0.019765,0.035329,0.006874,-0.006874,-0.037963,0.037963,0.020089,-0.018003,-0.002173


Pode-se notar que existem duas colunas categóricas que geram tal correlação: **Mental_Health_Condition** e **Consultation_History**, ambas que tem como opções apenas **Yes** e **No**. Portanto, pode-se remover uma dessas duas colunas numéricas geradas por cada campo (aqui escolheu-se remover as da opção **No**).

In [16]:
df.drop(['Mental_Health_Condition_No', 'Consultation_History_No'], axis=1, inplace=True)

Além disso, vale-se observar também as colunas com mais de duas opções. Caso as opções sejam compreensivas, de forma que não tenha como existir outra opção além das apresentadas na base de dados, pode-se remover uma das colunas - já que a abscência de valores em todas as outras implicaria, necessariamente, no preenchimento dela.

Não se pode dizer isso sobre a coluna **Gender** e **Occupation**, já que existem outras possibilidades de valores além das mencionadas. Entretanto, a **Occupation** e a **Country** cumprem este requisito, por causa da opção **Other**. Então, pode-se remover a coluna com esta opção. Além disso, a **Stress_Level** também cumpre: neste contexto, não há outra opção para o nível de estresse além de baixo (**Low**), médio (**Medium**) ou alto (**High**). Portanto, pode-se remover uma de suas colunas também (aqui removeu-se a do **High**).

In [17]:
df.drop(['Country_Other'], axis=1, inplace=True)
df.drop(['Occupation_Other'], axis=1, inplace=True)
df.drop(['Stress_Level_High'], axis=1, inplace=True)

Após as remoções, os campos do dataset ficaram como se segue.

In [18]:
df.head()

,Age,Sleep_Hours,Work_Hours,Physical_Activity_Hours,Gender_Female,Gender_Male,Gender_Non-binary,Gender_Prefer not to say,Occupation_Education,Occupation_Engineering,...,Country_Australia,Country_Canada,Country_Germany,Country_India,Country_UK,Country_USA,Mental_Health_Condition_Yes,Consultation_History_Yes,Stress_Level_Low,Stress_Level_Medium
0,36,7.1,46,5,False,False,True,False,False,False,...,False,True,False,False,False,False,False,True,False,True
1,34,7.5,47,8,True,False,False,False,True,False,...,False,False,False,False,True,False,True,False,True,False
2,65,8.4,58,10,False,False,True,False,False,False,...,False,False,False,False,False,True,True,False,True,False
3,34,9.8,30,2,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
4,22,4.9,62,5,True,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True


## Separando Dados de Treino e Teste

Após isso, pode-se separar os dados da base em dados de treinamento do modelo (que correspondem a 80% do total), e dados de teste (que correspondem a 20% do total).

In [46]:
train , test = train_test_split(df, train_size=0.8, random_state=64)

## Separando as Variáveis Alvo (Y) das Características (X)

Feito isso, pode-se separar o campo da variável alvo Y dos campos das características X. Neste caso, como se busca observar a tendência de condições de saúde mental, separa-se o campo **Mental_Health_Condition_Yes** como o Y, dos outros campos X.

In [47]:
train_x = train.drop(columns=['Mental_Health_Condition_Yes'], axis=1)
train_y = train['Mental_Health_Condition_Yes']

In [48]:
test_x = test.drop(columns=['Mental_Health_Condition_Yes'], axis=1)
test_y = test['Mental_Health_Condition_Yes']

Visualizando os resultados destes processos:

In [49]:
train_x.head()

,Age,Sleep_Hours,Work_Hours,Physical_Activity_Hours,Gender_Female,Gender_Male,Gender_Non-binary,Gender_Prefer not to say,Occupation_Education,Occupation_Engineering,...,Occupation_Sales,Country_Australia,Country_Canada,Country_Germany,Country_India,Country_UK,Country_USA,Consultation_History_Yes,Stress_Level_Low,Stress_Level_Medium
471,65,5.8,42,1,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
988,26,8.4,44,3,True,False,False,False,False,True,...,False,False,True,False,False,False,False,True,False,False
320,18,5.0,67,1,False,True,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
295,50,8.0,31,3,False,True,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
310,59,7.7,47,5,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,False


In [23]:
train_y.head()

,Mental_Health_Condition_Yes
799,True
471,True
242,False
704,True
839,True


In [24]:
test_x.head()

,Age,Sleep_Hours,Work_Hours,Physical_Activity_Hours,Gender_Female,Gender_Male,Gender_Non-binary,Gender_Prefer not to say,Occupation_Education,Occupation_Engineering,...,Occupation_Sales,Country_Australia,Country_Canada,Country_Germany,Country_India,Country_UK,Country_USA,Consultation_History_Yes,Stress_Level_Low,Stress_Level_Medium
632,55,8.9,67,6,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
687,23,7.5,72,3,False,False,False,True,False,False,...,True,False,False,False,False,False,True,True,False,True
833,60,7.5,77,0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
182,27,7.9,32,3,False,True,False,False,False,False,...,False,False,False,False,False,False,True,True,True,False
831,63,5.8,66,4,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,True,False


In [25]:
test_y.head()

,Mental_Health_Condition_Yes
632,False
687,False
833,False
182,False
831,True


## Treinando o Modelo

Nesta etapa, treina-se o modelo com os algorítimos supervisionados de classificação. Foram escolhidos seis algoritmos para o treinamento:

- Regressão Logística
- Análise Discriminante Linear
- Árvore de Decisão
- K Vizinhos Mais Próximos
- Máquinas de Vetores de Suporte
- Floresta Aleatória

In [50]:
LR  = LogisticRegression(solver='lbfgs', max_iter=1000).fit(train_x, train_y)
LDA = LinearDiscriminantAnalysis().fit(train_x, train_y)
DT  = DecisionTreeClassifier().fit(train_x, train_y)
KN  = KNeighborsClassifier().fit(train_x, train_y)
SVM = SVC().fit(train_x,train_y)
RF  = RandomForestClassifier().fit(train_x,train_y)

## Métricas da Matriz de Confusão

Feito o treinamento, pode-se obter as métricas de cada algoritmo com os dados de teste

In [51]:
metricas = {'Acurácia': [LR.score(test_x,test_y),
                         LDA.score(test_x,test_y),
                         DT.score(test_x,test_y),
                         KN.score(test_x,test_y),
                         SVM.score(test_x,test_y),
                         RF.score(test_x,test_y)],
            'Precisão': [precision_score(test_y, LR.predict(test_x)),
                      precision_score(test_y, LDA.predict(test_x)),
                      precision_score(test_y, DT.predict(test_x)),
                      precision_score(test_y, KN.predict(test_x)),
                      precision_score(test_y, SVM.predict(test_x)),
                      precision_score(test_y, RF.predict(test_x))],
            'Revocação' : [recall_score(test_y, RF.predict(test_x)),
                    recall_score(test_y, LDA.predict(test_x)),
                    recall_score(test_y, DT.predict(test_x)),
                    recall_score(test_y, KN.predict(test_x)),
                    recall_score(test_y, SVM.predict(test_x)),
                    recall_score(test_y, RF.predict(test_x))],
            'F1' : [f1_score(test_y, RF.predict(test_x)),
                    f1_score(test_y, LDA.predict(test_x)),
                    f1_score(test_y, DT.predict(test_x)),
                    f1_score(test_y, KN.predict(test_x)),
                    f1_score(test_y, SVM.predict(test_x)),
                    f1_score(test_y, RF.predict(test_x))]
        }

dados = pd.DataFrame(metricas,
                     columns = ['Acurácia', 'Precisão','Revocação','F1'],
                     index=['LR','LDA','DT','KNN','SVM','RF'])

dados

,Acurácia,Precisão,Revocação,F1
LR,0.510,0.520000,0.495146,0.470046
LDA,0.510,0.520000,0.631068,0.570175
DT,0.465,0.479167,0.446602,0.462312
KNN,0.425,0.442308,0.446602,0.444444
SVM,0.510,0.513228,0.941748,0.664384
RF,0.425,0.447368,0.495146,0.470046


## Análise de Resultados e Remodelagem

Como se pode ver, os modelos ficaram com uma média de 50% em todos os casos. Isso indica que existem muitos campos que não tem qualquer relação com a presença de condição de saúde mental nos entrevistados.

Um campo que pode estar nesta lista é o de ocupação. Embora sejam serviços de natureza diferentes, em termos físicos, as exigências de serviços de venda, educação, engenharia, entre outros mencionados são semelhantes ao corpo humano em natureza.

Analisando e removendo, portanto, os campos relacionados a eles:

In [52]:
print(train_x.columns)

Index(['Age', 'Sleep_Hours', 'Work_Hours', 'Physical_Activity_Hours',
       'Gender_Female', 'Gender_Male', 'Gender_Non-binary',
       'Gender_Prefer not to say', 'Occupation_Education',
       'Occupation_Engineering', 'Occupation_Finance', 'Occupation_Healthcare',
       'Occupation_IT', 'Occupation_Sales', 'Country_Australia',
       'Country_Canada', 'Country_Germany', 'Country_India', 'Country_UK',
       'Country_USA', 'Consultation_History_Yes', 'Stress_Level_Low',
       'Stress_Level_Medium'],
      dtype='object')


In [32]:
train_x = train_x.drop(columns=['Occupation_Education', 'Occupation_Engineering', 'Occupation_Finance', 'Occupation_Healthcare', 'Occupation_IT', 'Occupation_Sales'], axis=1)
test_x = test_x.drop(columns=['Occupation_Education', 'Occupation_Engineering', 'Occupation_Finance', 'Occupation_Healthcare', 'Occupation_IT', 'Occupation_Sales'], axis=1)
print(train_x.columns)

Index(['Age', 'Sleep_Hours', 'Work_Hours', 'Physical_Activity_Hours',
       'Gender_Female', 'Gender_Male', 'Gender_Non-binary',
       'Gender_Prefer not to say', 'Country_Australia', 'Country_Canada',
       'Country_Germany', 'Country_India', 'Country_UK', 'Country_USA',
       'Consultation_History_Yes', 'Stress_Level_Low', 'Stress_Level_Medium'],
      dtype='object')


Análogamente, os países também não parecem ter uma forte correlação por si próprios com a existência de uma condição de saúde mental ou não. Todos os listados são desenvolvidos ou emergentes, grandes e com bastante população. Removendo também estas colunas:

In [34]:
train_x = train_x.drop(columns=['Country_Australia', 'Country_Canada',
       'Country_Germany', 'Country_India', 'Country_UK', 'Country_USA'], axis=1)
test_x = test_x.drop(columns=['Country_Australia', 'Country_Canada',
       'Country_Germany', 'Country_India', 'Country_UK', 'Country_USA'], axis=1)
print(train_x.columns)

Index(['Age', 'Sleep_Hours', 'Work_Hours', 'Physical_Activity_Hours',
       'Gender_Female', 'Gender_Male', 'Gender_Non-binary',
       'Gender_Prefer not to say', 'Consultation_History_Yes',
       'Stress_Level_Low', 'Stress_Level_Medium'],
      dtype='object')


In [45]:
print(sum(train_y))

409


In [39]:
LR  = LogisticRegression(solver='lbfgs', max_iter=100).fit(train_x, train_y)
LDA = LinearDiscriminantAnalysis().fit(train_x, train_y)
DT  = DecisionTreeClassifier().fit(train_x, train_y)
KN  = KNeighborsClassifier().fit(train_x, train_y)
SVM = SVC().fit(train_x,train_y)
RF  = RandomForestClassifier().fit(train_x,train_y)

metricas = {'Acurácia': [LR.score(test_x,test_y),
                         LDA.score(test_x,test_y),
                         DT.score(test_x,test_y),
                         KN.score(test_x,test_y),
                         SVM.score(test_x,test_y),
                         RF.score(test_x,test_y)],
            'Precisão': [precision_score(test_y, LR.predict(test_x)),
                      precision_score(test_y, LDA.predict(test_x)),
                      precision_score(test_y, DT.predict(test_x)),
                      precision_score(test_y, KN.predict(test_x)),
                      precision_score(test_y, SVM.predict(test_x)),
                      precision_score(test_y, RF.predict(test_x))],
            'Revocação' : [recall_score(test_y, RF.predict(test_x)),
                    recall_score(test_y, LDA.predict(test_x)),
                    recall_score(test_y, DT.predict(test_x)),
                    recall_score(test_y, KN.predict(test_x)),
                    recall_score(test_y, SVM.predict(test_x)),
                    recall_score(test_y, RF.predict(test_x))],
            'F1' : [f1_score(test_y, RF.predict(test_x)),
                    f1_score(test_y, LDA.predict(test_x)),
                    f1_score(test_y, DT.predict(test_x)),
                    f1_score(test_y, KN.predict(test_x)),
                    f1_score(test_y, SVM.predict(test_x)),
                    f1_score(test_y, RF.predict(test_x))]
        }

dados = pd.DataFrame(metricas,
                     columns = ['Acurácia', 'Precisão','Revocação','F1'],
                     index=['LR','LDA','DT','KNN','SVM','RF'])

dados

,Acurácia,Precisão,Revocação,F1
LR,0.535,0.553719,0.537736,0.515837
LDA,0.540,0.558333,0.632075,0.592920
DT,0.460,0.490000,0.462264,0.475728
KNN,0.485,0.514563,0.500000,0.507177
SVM,0.520,0.530488,0.820755,0.644444
RF,0.465,0.495652,0.537736,0.515837


## Conclusões

Tendo em vista as métricas apresentadas sobre os modelos treinados, algumas das principais conclusões que pode-se obter são:
- O treinamento por ??? é o melhor, pois...
- Os campos que mais interferem no resultado final são...